In [1]:
!pip install transformers pandas scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip3 install torch torchvision torchaudio
!pip install datasets


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/DialoGPT-medium"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/Users/diego/Desktop/counselChatModel/caldito/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### counsel chat dataset training

In [5]:
from datasets import load_dataset

counsel_chat_ds = load_dataset("nbertagnolli/counsel-chat")

Repo card metadata block was not found. Setting CardData to empty.


In [10]:
import pandas as pd

counsel_chat_df = pd.DataFrame(counsel_chat_ds)
print(counsel_chat_df.head())
print(counsel_chat_df['train'][0])

                                               train
0  {'questionID': 0, 'questionTitle': 'Do I have ...
1  {'questionID': 0, 'questionTitle': 'Do I have ...
2  {'questionID': 0, 'questionTitle': 'Do I have ...
3  {'questionID': 0, 'questionTitle': 'Do I have ...
4  {'questionID': 0, 'questionTitle': 'Do I have ...
{'questionID': 0, 'questionTitle': 'Do I have too many issues for counseling?', 'questionText': 'I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?', 'questionLink': 'https://counselchat.com/questions/do-i-have-too-many-issues-for-counseling', 'topic': 'depression', 'therapistInfo': 'Jennifer MolinariHypnotherapist & Licensed Counselor', 'therapistURL': 

In [ ]:
print(counsel_chat_df['train'][0]["questionTitle"])

# removing keys from the dataset that are not needed
for i in range(len(counsel_chat_df)):
    counsel_chat_df["train"][i].pop("questionID")
    counsel_chat_df["train"][i].pop("questionLink")
    counsel_chat_df["train"][i].pop("therapistInfo")
    counsel_chat_df["train"][i].pop("therapistURL")
    counsel_chat_df["train"][i].pop("upvotes")
    counsel_chat_df["train"][i].pop("views")

Do I have too many issues for counseling?


In [25]:
print(counsel_chat_df["train"][98])

{'questionTitle': 'How can I help my girlfriend?', 'questionText': 'My girlfriend just quit drinking and she became really depressed. She told me that she wants to move. What can I do to help her? I want her to stay.', 'topic': 'depression', 'answerText': 'I would say talk to her about her depression. The best thing you can do is let her know that you are there for her as a support system. I would encourage her to talk more if it helps and encourage her to get move involved in doing activities she likes to do. Also, getting connected in a community of people is important to help with feeling depressed. Encourage her to speak with a therapist or counselor to address her depression as she has made a major change in her life by deciding to stop drinking. She may believe moving is the best to help with this. You can share with her that you want her to stay and express to her how you feel. Ultimately, you will have to respect her boundaries and the decision she chooses to make. She may want

In [61]:
# convert dataset to conversation pairs
def preprocess_data(dataset):
    all_samples = []
    
    conversation = dataset["train"]
    
    for i in range(len(conversation)):
        question = conversation[i]["questionText"]
        answer = conversation[i]["answerText"]
        question_title = conversation[i]["questionTitle"]
        topic = conversation[i]["topic"]
        
        all_samples.append({
            "question_title": question_title,
            "topic": topic,
            "conversation": [
                {"question": question},
                {"answer": answer}
            ]
        })
        
    return all_samples

new_cc_dataset = preprocess_data(counsel_chat_df)
for i in range(0, 6):
    print(new_cc_dataset[i])
    
print(f"\n{new_cc_dataset[0]["conversation"][0]["question"]}")

{'question_title': 'Do I have too many issues for counseling?', 'topic': 'depression', 'conversation': [{'question': 'I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?'}, {'answer': 'It is very common for\xa0people to have multiple issues that they want to (and need to) address in counseling.\xa0 I have had clients ask that same question and through more exploration, there is often an underlying fear that they\xa0 "can\'t be helped" or that they will "be too much for their therapist." I don\'t know if any of this rings true for you. But, most people have more than one problem in their lives and more often than not,\xa0 people have numerous significant stressors in 

In [109]:
# flatten list of samples
def prepare_data_for_trainer(all_samples):
    samples = []
    
    for sample in all_samples:
        question_text = str(sample["conversation"][0]["question"])
        answer_text = str(sample["conversation"][1]["answer"])
        
        # replace the non-breaking space character
        answer_text = answer_text.replace("\xa0", " ")
        # fix escaped quotes
        answer_text = answer_text.replace(r"\'", "'")
        # replace multiple spaces with a single space
        answer_text = " ".join(answer_text.split())
        
        prompt_with_metadata = f"Question Title: {sample["question_title"]}, Topic: {sample["topic"]}, Question: {question_text}"
        
        samples.append({
            "prompt": prompt_with_metadata,
            "response": answer_text
        })
        
    return samples

all_samples = prepare_data_for_trainer(new_cc_dataset)
print(all_samples[0])

{'prompt': 'Question Title: Do I have too many issues for counseling?, Topic: depression, Question: I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac. I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years. I’ve never had counseling about any of this. Do I have too many issues to address in counseling?', 'response': 'It is very common for people to have multiple issues that they want to (and need to) address in counseling. I have had clients ask that same question and through more exploration, there is often an underlying fear that they "can\'t be helped" or that they will "be too much for their therapist." I don\'t know if any of this rings true for you. But, most people have more than one problem in their lives and more often than not, people have numerous significant stressors in their lives. Let\'s face it, life can be

In [111]:
# split the data
# in this case the dataset only has a training split without separate test or validation splits.
# here i created these splits myself using scikit-learn
from sklearn.model_selection import train_test_split

# train and test splits
train_samples, test_samples = train_test_split(all_samples, test_size=0.1, random_state=42)
# creating a validation split from the training data
train_samples, val_samples = train_test_split(train_samples, test_size=0.1, random_state=42)

print(f"Training samples: {len(train_samples)}")
print(f"Validation samples: {len(val_samples)}")
print(f"Test samples: {len(test_samples)}")

Training samples: 2247
Validation samples: 250
Test samples: 278


In [118]:
# convert to hugging face Datasets
from datasets import Dataset
print(type(train_samples))

cc_train_dataset = Dataset.from_list(train_samples)
cc_val_dataset = Dataset.from_list(val_samples)
cc_test_dataset = Dataset.from_list(test_samples)

<class 'list'>


In [123]:
tokenizer.pad_token = tokenizer.eos_token

# tokenization function
def tokenize_cc_data(samples):
    # combine prompt and response for training
    inputs = samples["prompt"]
    targets = samples["response"]
    
    combined_texts = [f"{input_text}\n{target_text}" for input_text, target_text in zip(inputs, targets)]
    
    # tokenize with padding and truncation
    tokenized_outputs = tokenizer(
        combined_texts,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    
    tokenized_outputs["labels"] = tokenized_outputs["input_ids"].clone()
    
    return tokenized_outputs

tokenized_cc_train = cc_train_dataset.map(tokenize_cc_data, batched=True)
tokenized_cc_val = cc_val_dataset.map(tokenize_cc_data, batched=True)
tokenized_cc_test = cc_test_dataset.map(tokenize_cc_data, batched=True)

Map: 100%|██████████| 278/278 [00:00<00:00, 2863.08 examples/s]


In [ ]:
from transformers import TrainingArguments, Trainer

# define training arguments
training_args = TrainingArguments(
    output_dir="./cc_results",         # directory to save model checkpoints
    num_train_epochs=3,                 # adjust based on convergence
    per_device_train_batch_size=8,      # increase to 16 if vram allows, or decrease to 8
    per_device_eval_batch_size=8, 
    gradient_accumulation_steps=2,      # helps if batch size is small
    evaluation_strategy="epoch",        # evaluate at the end of each epoch
    save_strategy="epoch",              # save model checkpoints each epoch
    logging_dir="./logs/cc_logs",               # logging directory
    logging_steps=100,                  # adjust based on dataset size
    learning_rate=5e-5,                 # starndard for transformer fine-tuning
    warmup_steps=500,                   # helps stabilize training
    weight_decay=0.01,                  # regularization
    bf16=True,                          # enable mixed precision for speedup
    save_total_limit=3,                 # keep last 3 checkpoints
    eval_accumulation_steps=4,          # accumuluate loss over multiple steps
    logging_first_step=True,            # log first step
)

# create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_cc_train,
    eval_dataset=tokenized_cc_val,
    tokenizer=tokenizer,
)